<a href="https://colab.research.google.com/github/Alphabf/CDISC-Implementation/blob/main/Generate_ADSL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
library(glue)
library(tidyverse)
library(haven)
library(assertthat)
library(huxtable)
library(data.table)
library(lubridate)
library(pharmaRTF)

In [ ]:
load("Data_Need_forADSL.RData")
ls()

[1] "dm"     "ds"     "suppdm" "vs"

**Merge DM and SUPPDM**

Transponse SUPPDM data such that each QNAM becomes a variable

In [ ]:
suppdm01 <- suppdm %>%
    mutate(qnam=str_to_lower(qnam)) %>%
    group_by(studyid, usubjid) %>%
    pivot_wider(id_cols=c(studyid,usubjid),names_from = qnam, values_from = qval)

dm01 <- left_join(dm, suppdm01, by = c("studyid", "usubjid"))

In [ ]:
dm01

**Create variables that are directly based on input variables**

In [ ]:
adsl01 <- dm01 %>%

    # Planned and actual treatment values;

mutate(trt01p = if_else(armcd == "PBO", "Placebo", if_else(armcd == "ACTIVE", "Active", "")),
       trt01a = if_else(actarmcd == "PBO", "Placebo", if_else(actarmcd == "ACTIVE", "Active", "")),
       trt01pn = if_else(armcd == "PBO", 1, if_else(armcd == "ACTIVE", 2, NA)),
       trt01an = if_else(actarmcd == "PBO", 1, if_else(actarmcd == "ACTIVE", 2, NA))) %>%

    # Date conversions

mutate(tr01sdt = as.Date(rfxstdtc),
tr01edt = as.Date(rfxendtc),
rficdt = as.Date(rficdtc),
lstalvdt = as.Date(rfpendtc),
dthdt = as.Date(dthdtc),
trtsdt = tr01sdt,
trtedt = tr01edt) %>%

    # Grouping variables
mutate(agegr1 = if_else(!is.na(age) & age < 60, "< 60 Years",
if_else(age >= 60, ">= 60 Years", "")),
agegr1n = if_else(!is.na(age) & age < 60, 1, if_else(age >= 60, 2,NA)))

In [ ]:
 select(adsl01, c('usubjid', 'trt01p','trt01a','trt01pn', 'trt01an','agegr1','agegr1n', 'ethnic'))

usubjid,trt01p,trt01a,trt01pn,trt01an,agegr1,agegr1n,ethnic
<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<chr>
CSG001-1001,,,NA,NA,< 60 Years,1,HISPANIC OR LATINO
CSG001-1002,,,NA,NA,< 60 Years,1,NOT HISPANIC OR LATINO
CSG001-1003,Placebo,,1,NA,< 60 Years,1,HISPANIC OR LATINO
CSG001-1004,Active,Active,2,2,< 60 Years,1,HISPANIC OR LATINO
CSG001-1005,Active,Placebo,2,1,>= 60 Years,2,NOT HISPANIC OR LATINO
CSG001-1006,Placebo,Placebo,1,1,>= 60 Years,2,NOT HISPANIC OR LATINO
CSG001-1007,Placebo,Placebo,1,1,< 60 Years,1,NOT HISPANIC OR LATINO
CSG001-1008,Active,Active,2,2,>= 60 Years,2,NOT HISPANIC OR LATINO


**Process disposition data for treatment and study status**  
      
Enrollment

In [ ]:
enrl01 <- ds %>%
filter(dsterm == "ENROLLED") %>%
mutate(enrldt = as.Date(dsstdtc)) %>%
select(studyid, usubjid, enrldt)

Randomization

In [ ]:
rand01 <- ds %>%
filter(dsterm == "RANDOMIZED") %>%
mutate(randdt = as.Date(dsstdtc)) %>%
select(studyid, usubjid, randdt)

End of treatment

In [ ]:
eot01 <- ds %>%
 filter(dsscat == "END OF TREATMENT") %>%
 mutate(
 eotstt = case_when(
 dsdecod == "COMPLETED" ~ "COMPLETED",
 dsdecod != "" ~ "DISCONTINUED",
 TRUE ~ NA
 ),
 dctreas = if_else(dsdecod != "COMPLETED", dsdecod, NA),
 ineot = 1
 ) %>%
 select(studyid, usubjid, eotstt, dctreas, ineot)


                 
End of study;

In [ ]:
eos01 <- ds %>%
 filter(dsscat == "END OF STUDY") %>%
 mutate(
 eosstt = case_when(
 dsdecod == "COMPLETED" ~ "COMPLETED",
 dsdecod != "" ~ "DISCONTINUED",
 TRUE ~ NA
 ),
 dcsreas = if_else(dsdecod != "COMPLETED", dsdecod, NA),
 eosdt = as.Date(dsstdtc),
 ineos = 1
 ) %>%
 select(studyid, usubjid, eosstt, dcsreas, eosdt, ineos)

Baseline variables from vital signs;

In [ ]:
heightbl <- vs %>%
 filter(vsblfl == "Y" & vstestcd == "HEIGHT") %>%
 select(studyid, usubjid, vsstresn) %>%
 rename(heightbl = vsstresn)

weightbl <- vs %>%
 filter(vsblfl == "Y" & vstestcd == "WEIGHT") %>%
 select(studyid, usubjid, vsstresn) %>%
 rename(weightbl = vsstresn)



Bring disposition and vital signs related info into parent dataset;

In [ ]:
library(purrr)

data_frames <- list(adsl01, eot01, eos01, enrl01, rand01, heightbl, weightbl)

adsl02 <- reduce(data_frames, left_join, by = c("studyid", "usubjid"))

Create variables/assign values to existing variables which are dependent on their variables;

In [ ]:

adsl03 <- adsl02 %>%
 mutate(
 saffl = if_else(!is.na(trtsdt), "Y", "N"),
 randfl = if_else(!is.na(randdt), "Y", "N"),
 enrlfl = if_else(!is.na(enrldt), "Y", "N"),
 complfl = if_else(eosstt == "COMPLETED", "Y", "N", "N"),
 eotstt = if_else((eotstt == "" | is.na(eotstt))  & is.na(ineot) & saffl == "Y", "ONGOING", eotstt),
 eosstt = if_else((eosstt == "" | is.na(eosstt)) & is.na(ineos) & saffl == "Y", "ONGOING", eosstt),
 trtdurd = if_else(!is.na(trtsdt) & !is.na(trtedt), trtedt - trtsdt + 1, NA),
 trtdurd = as.numeric(trtdurd)
 )

In [ ]:
adsl03 <- adsl03 %>%
 mutate(across(where(is.character), ~if_else(is.na(.), "", .)))

Keep only required variables;

In [ ]:
varlist <- c("STUDYID", "USUBJID", "SUBJID", "SITEID", "AGE", "AGEU", "AGEGR1", "AGEGR1N", "SEX", "RACE", "RACE1", "RACE2", "RACESP", "SAFFL", "COMPLFL", "RANDFL", "ENRLFL", "ARM", "ACTARM", "TRT01P", "TRT01PN", "TRT01A", "TRT01AN", "TRTSDT", "TRTEDT", "TR01SDT", "TR01EDT", "EOSSTT", "EOSDT", "DCSREAS", "EOTSTT", "DCTREAS", "RFICDT", "ENRLDT", "RANDDT", "LSTALVDT", "TRTDURD", "DTHDT", "HEIGHTBL", "WEIGHTBL")

adsl <- adsl03 %>%
 rename_all(toupper) %>%
 select(all_of(varlist))

output<-adsl

In [ ]:
output

STUDYID,USUBJID,SUBJID,SITEID,AGE,AGEU,AGEGR1,AGEGR1N,SEX,RACE,⋯,EOTSTT,DCTREAS,RFICDT,ENRLDT,RANDDT,LSTALVDT,TRTDURD,DTHDT,HEIGHTBL,WEIGHTBL
<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,⋯,<chr>,<chr>,<date>,<date>,<date>,<date>,<dbl>,<date>,<dbl>,<dbl>
CSG001,CSG001-1001,1001,10,35,YEARS,< 60 Years,1,M,WHITE,⋯,,,2010-01-01,NA,NA,2010-01-01,NA,NA,NA,NA
CSG001,CSG001-1002,1002,10,40,YEARS,< 60 Years,1,F,MULTIPLE,⋯,,,2010-01-01,2010-01-04,NA,2010-01-05,NA,NA,NA,NA
CSG001,CSG001-1003,1003,10,40,YEARS,< 60 Years,1,M,OTHER,⋯,,,2010-01-01,2010-01-03,2010-01-03,2010-01-05,NA,2010-01-05,NA,NA
CSG001,CSG001-1004,1004,10,38,YEARS,< 60 Years,1,M,WHITE,⋯,COMPLETED,,2010-01-01,2010-01-04,2010-01-05,2010-02-28,21,NA,177.00,87.3
CSG001,CSG001-1005,1005,10,64,YEARS,>= 60 Years,2,M,AMERICAN INDIAN OR ALASKA NATIVE,⋯,ONGOING,,2010-01-15,2010-02-01,2010-02-05,2020-02-20,8,NA,66.14,76.1
CSG001,CSG001-1006,1006,10,75,YEARS,>= 60 Years,2,F,NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER,⋯,DISCONTINUED,ADVERSE EVENT,2010-02-18,2010-03-01,2010-03-01,2010-03-25,9,NA,160.00,60.9
CSG001,CSG001-1007,1007,10,32,YEARS,< 60 Years,1,M,UNKNOWN,⋯,COMPLETED,,2010-04-04,2010-04-14,2010-04-14,2010-06-12,22,NA,178.00,85.4
CSG001,CSG001-1008,1008,10,83,YEARS,>= 60 Years,2,F,NOT REPORTED,⋯,DISCONTINUED,SUBJECT REQUEST,2010-06-20,2010-06-26,2010-06-27,2010-08-18,15,NA,NA,NA


In [ ]:
save(adsl, file = "C:/Users/Waraba/Desktop/Ckinical Trial Training Materiels/ADaM_withR/ADaM_ADSL_withR/ADaM_ADSL_L1101/adsl.RData")

In [ ]:
adsl

STUDYID,USUBJID,SUBJID,SITEID,AGE,AGEU,AGEGR1,AGEGR1N,SEX,RACE,⋯,EOTSTT,DCTREAS,RFICDT,ENRLDT,RANDDT,LSTALVDT,TRTDURD,DTHDT,HEIGHTBL,WEIGHTBL
<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,⋯,<chr>,<chr>,<date>,<date>,<date>,<date>,<dbl>,<date>,<dbl>,<dbl>
CSG001,CSG001-1001,1001,10,35,YEARS,< 60 Years,1,M,WHITE,⋯,,,2010-01-01,NA,NA,2010-01-01,NA,NA,NA,NA
CSG001,CSG001-1002,1002,10,40,YEARS,< 60 Years,1,F,MULTIPLE,⋯,,,2010-01-01,2010-01-04,NA,2010-01-05,NA,NA,NA,NA
CSG001,CSG001-1003,1003,10,40,YEARS,< 60 Years,1,M,OTHER,⋯,,,2010-01-01,2010-01-03,2010-01-03,2010-01-05,NA,2010-01-05,NA,NA
CSG001,CSG001-1004,1004,10,38,YEARS,< 60 Years,1,M,WHITE,⋯,COMPLETED,,2010-01-01,2010-01-04,2010-01-05,2010-02-28,21,NA,177.00,87.3
CSG001,CSG001-1005,1005,10,64,YEARS,>= 60 Years,2,M,AMERICAN INDIAN OR ALASKA NATIVE,⋯,ONGOING,,2010-01-15,2010-02-01,2010-02-05,2020-02-20,8,NA,66.14,76.1
CSG001,CSG001-1006,1006,10,75,YEARS,>= 60 Years,2,F,NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER,⋯,DISCONTINUED,ADVERSE EVENT,2010-02-18,2010-03-01,2010-03-01,2010-03-25,9,NA,160.00,60.9
CSG001,CSG001-1007,1007,10,32,YEARS,< 60 Years,1,M,UNKNOWN,⋯,COMPLETED,,2010-04-04,2010-04-14,2010-04-14,2010-06-12,22,NA,178.00,85.4
CSG001,CSG001-1008,1008,10,83,YEARS,>= 60 Years,2,F,NOT REPORTED,⋯,DISCONTINUED,SUBJECT REQUEST,2010-06-20,2010-06-26,2010-06-27,2010-08-18,15,NA,NA,NA


In [ ]:
colnames(adsl)

[1] "STUDYID"  "USUBJID"  "SUBJID"   "SITEID"   "AGE"      "AGEU"    
 [7] "AGEGR1"   "AGEGR1N"  "SEX"      "RACE"     "RACE1"    "RACE2"   
[13] "RACESP"   "SAFFL"    "COMPLFL"  "RANDFL"   "ENRLFL"   "ARM"     
[19] "ACTARM"   "TRT01P"   "TRT01PN"  "TRT01A"   "TRT01AN"  "TRTSDT"  
[25] "TRTEDT"   "TR01SDT"  "TR01EDT"  "EOSSTT"   "EOSDT"    "DCSREAS" 
[31] "EOTSTT"   "DCTREAS"  "RFICDT"   "ENRLDT"   "RANDDT"   "LSTALVDT"
[37] "TRTDURD"  "DTHDT"    "HEIGHTBL" "WEIGHTBL"